# Superhero (and Supervillain) Name Generator

---

[Superhero Names Dataset](https://github.com/am1tyadav/superhero)

## Task 2

1. Import the data
2. Create a tokenizer
3. Char to index and Index to char dictionaries

In [7]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

tf.__version__

'2.15.0'

In [1]:
!git clone https://github.com/am1tyadav/superhero

Cloning into 'superhero'...
remote: Enumerating objects: 8, done.
remote: Counting objects: 100% (8/8), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 8 (delta 0), reused 4 (delta 0), pack-reused 0
Receiving objects: 100% (8/8), 47.08 KiB | 5.23 MiB/s, done.


In [14]:
with open('superhero/superheroes.txt', 'r') as f:
    data = f.read()

In [15]:
data[:100]

'jumpa\t\ndoctor fate\t\nstarlight\t\nisildur\t\nlasher\t\nvarvara\t\nthe target\t\naxel\t\nbattra\t\nchangeling\t\npyrrh'

In [16]:
tokenizer = tf.keras.preprocessing.text.Tokenizer(
    filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~',
    split='\n',
)

In [17]:
tokenizer.fit_on_texts(data)

In [18]:
char_to_index = tokenizer.word_index
index_to_char = dict((i, c) for c, i in char_to_index.items())

In [ ]:
index_to_char

## Task 3

1. Converting between names and sequences

In [ ]:
names = data.splitlines()
names[:10]

In [23]:
tokenizer.texts_to_sequences(names[0])

[[25], [16], [12], [20], [2], [1]]

In [24]:
def name_to_seq(name):
  return [tokenizer.texts_to_sequences(c)[0][0] for c in name]

In [25]:
name_to_seq(names[0])

[25, 16, 12, 20, 2, 1]

In [37]:
def seq_to_name(seq):
  return ''.join([index_to_char[i] for i in seq])

In [38]:
seq_to_name([25, 16, 12, 20, 2, 1])

'jumpa\t'

## Task 4

1. Creating sequences
2. Padding all sequences

In [42]:
sequences = []

for name in names:
  seq = name_to_seq(name)
  if len(seq) >= 2:
    sequences += [seq[:i] for i in range(2, len(seq) + 1)]

In [ ]:
sequences[:10]

In [43]:
max_seq_len = max([len(seq) for seq in sequences])
max_seq_len

33

In [44]:
padded_sequences = tf.keras.preprocessing.sequence.pad_sequences(
    sequences,
    maxlen = max_seq_len,
    padding = 'pre'
)

In [45]:
x, y = padded_sequences[:, :-1], padded_sequences[:, -1]

In [80]:
print(padded_sequences[2])
print(x[2], y[2])

[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0 25 16 12 20]
[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0 25 16 12] 20


In [46]:
from sklearn.model_selection import train_test_split

x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.2, random_state=42)

## Task 5: Creating Training and Validation Sets

1. Creating training and validation sets

In [47]:
num_chars = len(char_to_index.keys()) + 1
num_chars

29

In [51]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, MaxPool1D, Dense, Conv1D, Bidirectional
from tensorflow.keras.optimizers import Adam

In [54]:
model = Sequential([
    Embedding(num_chars, 10, input_length=max_seq_len - 1),
    Conv1D(64, 5, strides = 1, activation = 'tanh', padding = 'causal'),
    MaxPool1D(2),
    Bidirectional(LSTM(32)),
    Dense(num_chars, activation = 'softmax')
])

model.compile(loss = 'sparse_categorical_crossentropy',
              optimizer = Adam(learning_rate = 0.001),
              metrics = ['accuracy'])

model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, 32, 10)            290       
                                                                 
 conv1d_4 (Conv1D)           (None, 32, 64)            3264      
                                                                 
 max_pooling1d_2 (MaxPoolin  (None, 16, 64)            0         
 g1D)                                                            
                                                                 
 bidirectional_4 (Bidirecti  (None, 64)                24832     
 onal)                                                           
                                                                 
 dense_4 (Dense)             (None, 29)                1885      
                                                                 
Total params: 30271 (118.25 KB)
Trainable params: 3027

## Task 6: Creating the Model

## Task 7: Training the Model

In [ ]:
h = model.fit(x_train, y_train,
    validation_data=(x_val, y_val),
    epochs=10, verbose = 2,

    callbacks = [tf.keras.callbacks.EarlyStopping(monitor = 'val_accuracy', patience = 3)])

## Task 8: Generate Names!

In [ ]:
epochs = len(h.history['accuracy'])

plt.plot(0, epochs, h.history['accuracy'], label = 'Training Accuracy')
plt.plot(0, epochs, h.history['val_accuracy'], label = 'Validation Accuracy')
plt.legend()
plt.show()

In [64]:
def generate_names(seed):
  for i in range(0, 40):
    seq = name_to_seq(seed)
    padded_seq = tf.keras.preprocessing.sequence.pad_sequences(
        [seq],
        maxlen = max_seq_len - 1,
        padding = 'pre'
    )
    pred = model.predict(padded_seq)[0]
    pred_char = index_to_char[tf.argmax(pred).numpy()]
    seed += pred_char
    if pred_char == '\t':
      return seed

In [72]:
generate_names('pate')

1/1 [==============================] - 0s 26ms/step


'pater stark\t'